# Performance test of $two \ level \ model$.

$two \ level \ model$ is composed of first set of OVR classifiers $f_{c, other}$ and secnd set of OVR classifiers $f^{(2)}_{c, other}$ .  
We test the model using a classification task, which is compared to the results of a baseline model that uses only $f_{c, other}$ .

### Fix typo of fgcv->fgvc

In [20]:
import glob, os

In [21]:
for f in glob.iglob("trained_model/modelfgcv*"):
    os.rename(f, "trained_model/modelfgvc" + f[len('trained_model/modelfgcv'):])

## Set up

In [1]:
import os
import sys

import numpy as np

import pandas as pd
import glob
import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
from models.modelutils import dir2filedict_sorted
import random

Using TensorFlow backend.


Load category and file path information.

In [3]:
testdict = dir2filedict_sorted("data_fgvc/test")
categories = [str(i) for i in range(0, 100)]

Create (category, path) information of test datasets.

In [4]:
#for debug
# org_testdict = testdict
# categories = [str(i) for i in range(0, 3)]
# testdict = {key: org_testdict[key] for key in categories}

In [4]:
testtup = [(key, file) for key in categories for file in testdict[key] ]

In [5]:
len(testtup)

3333

Check the list.

In [6]:
testtup[0:5]

[('0', 'data_fgvc/test/0/0062765.jpg'),
 ('0', 'data_fgvc/test/0/0064932.jpg'),
 ('0', 'data_fgvc/test/0/0197342.jpg'),
 ('0', 'data_fgvc/test/0/0447936.jpg'),
 ('0', 'data_fgvc/test/0/0536515.jpg')]

Convert the list into a Pandas DataFrame for simple treatments.

In [7]:
testdf = pd.DataFrame({"category": [tup[0] for tup in testtup], "files": [tup[1] for tup in testtup]})

In [8]:
testdf_shuffled=testdf.sample(frac=1, random_state=123).reset_index(drop=True)

In [9]:
testdf_shuffled.head()

,category,files
0,13,data_fgvc/test/13/2233262.jpg
1,72,data_fgvc/test/72/1372357.jpg
2,53,data_fgvc/test/53/0773531.jpg
3,39,data_fgvc/test/39/1340322.jpg
4,13,data_fgvc/test/13/1313993.jpg


Define classes for $two \ level \ model$ .

In [10]:
from models.modelutils import load_best_model_if_exist
import os

In [11]:
class ModelBinder:
    def __init__(self, base_model_name, basedir, cats):
        self.base_model_name = base_model_name
        self.basedir = basedir
        self._models = {}
        self.verbose = True
        self._categories = cats
        self._OTHER_LABEL = "other"

    @classmethod
    def dup_from(cls, binder):
        return ModelBinder(binder.base_model_name, binder.basedir, binder._categories)

    def model_path(self, target_key):
        return os.path.join(self.basedir, "{}_{}".format(self.base_model_name, target_key))

    def get_or_load_model(self, target_key):
        if target_key in self._models:
            return self._models[target_key]
        self.notify("load {}".format(target_key))
        self._models[target_key] = load_best_model_if_exist(self.model_path(target_key))
        return self._models[target_key]

    def notify(self, msg):
        if self.verbose:
            print(msg)

    def load_all_models(self, keys):
        list(map(self.get_or_load_model, keys))
        
    def predict_one_with_log(self, key, arrs):
        self.notify("predict {}.".format(key))
        return self._models[key].predict(arrs)
        
    def predict_arrs(self, arrs):
        models = self._models
        preddict = {key: self.predict_one_with_log(key, arrs)[:, 1] for key in models.keys()}
        return pd.DataFrame(preddict)
    
    def _row2class(self, rowdf, threshold):
        for cat in self._categories:
            if rowdf[cat] >= threshold:
                return cat
        return self._OTHER_LABEL

    def df2classes(self, df, threshold = 0.5):
        res = []
        for i in range(len(df)):
            rowdf = df.iloc[i, :]
            res.append(self._row2class(rowdf, threshold))
        return res

In [12]:
class TwoLevelModel:
    def __init__(self, categories, h1binder, otherlabel="OTHER"):
        self._categories = categories
        self._h1binder = h1binder
        self._OTHERCLASS = -1
        self._OTHERLABEL = otherlabel
        # initial trial.
        self._FIRST_THRESHOLD = 0.8
        self._SECOND_THRESHOLD = 0.5
        self.verbose = True
        
    def notify(self, msg):
        if self.verbose:
            print(msg)

    def load_all(self):
        catkeys = self._categories
        self._h1binder.load_all_models(["sec_" + cat for cat in catkeys])

    def predict_arrs(self, arrs, firstdf):
        df = self._predict_arrs(arrs, firstdf)
        self._df = df
        return self._h1binder.df2classes(df, self._SECOND_THRESHOLD)

    def _predict_arrs(self, arrs, firstdf):
        # firstdf = pd.DataFrame(self._h2binder.predict_arrs(arrs))

        resultdf = pd.DataFrame(np.zeros(firstdf.shape))
        resultdf.columns = firstdf.columns

        for targetkey in self._categories:

            df = self._predict_second(targetkey, arrs, firstdf)
            if df is not None:
                resultdf.loc[df['orgindex'], targetkey] = df[targetkey].values

        return resultdf

    def _predict_second(self, targetcat, arrs, firstdf):
        filtered = firstdf[firstdf[targetcat] > self._FIRST_THRESHOLD]

        if len(filtered) == 0:
            return None

        farrs = arrs[filtered.index, :]
        model = self._h1binder.get_or_load_model('sec_' + targetcat)

        # no second level classifier, all score is already enough.
        if model == None:
            return pd.DataFrame({targetcat: np.ones(len(filtered.index)), 'orgindex': filtered.index})

        self.notify("predict {}".format('sec_' + targetcat))
        res = model.predict(farrs)
        scores = res[:, 1]

        return pd.DataFrame({targetcat: scores, 'orgindex': filtered.index})

In [13]:
from models.processor import DataSet

In [14]:
ds = DataSet()

## Conduct first level prediction (h2 prediction) and store result.

Also evaluate baseline model.

In [22]:
binder = ModelBinder( "modelfgvc", "trained_model", categories)

In [23]:
binder.load_all_models(categories)

load 0
load 1
load 2


### Evaluate baseline model

Classifiers are trained in *train.ipynb*.

In [16]:
#for debug
# org_testdf_shuffled = testdf_shuffled
# testdf_shuffled = org_testdf_shuffled[0:5]
# testdf_shuffled.shape

(5, 2)

In [61]:
#for debug
# modelsbackup = binder._models
# binder._models = {key: modelsbackup[key] for key in categories}

In [24]:
all_labels = pd.Series(dtype=object)
all_results = []
all_result_dfs = []

for chunk in tqdm.tqdm(ds.chunked(testdf_shuffled, 500)):
    x = ds.files_to_dataset(chunk['files'])
    label = chunk['category']
    resdf = binder.predict_arrs(x)
    # to compare two level model, we use threshold 0.8 for initial trial.
    resclasses = binder.df2classes(resdf, threshold =0.8)
    
    all_labels = all_labels.append(label,  ignore_index=True)
    all_results.extend(resclasses)
    all_result_dfs.append(resdf)

  0%|          | 0/1 [00:00<?, ?it/s]

predit 0.
predit 1.
predit 2.


100%|██████████| 1/1 [00:06<00:00,  6.55s/it]


In [66]:
print("Number of test images: {}\nAccuracy: {}".format(len(all_labels), float(sum(all_labels == all_results))/len(all_labels)))

Number of test images: 5
Accuracy: 0.2


Save the results as picke.

In [67]:
results_df = pd.DataFrame({"labels": all_labels, "prediction": all_results})

In [68]:
results_df.to_pickle("results/baseline_model_prediction_fgvc.dat")

In [69]:
h2df = pd.concat(all_result_dfs)
h2df.index = testdf_shuffled.index
h2df['files'] = testdf_shuffled["files"]

In [71]:
h2df.to_pickle("results/baseline_h2_df_fgvc.dat")

## Evaluate $two \ level \ model$

Classifiers are trained in *train_second.ipynb*.

In [15]:
h2df = pd.read_pickle("results/baseline_h2_df_fgvc.dat")

In [ ]:
# Fix of corrupted files column in h2df due to first version of bug (not necessary if you don't use old h2df dat)
h2df.index = testdf_shuffled.index
h2df['files'] = testdf_shuffled["files"]

In [216]:
sec_binder = ModelBinder( "modelfgvc", "trained_model", categories)

In [17]:
two_level_model = TwoLevelModel(categories, sec_binder)

In [18]:
two_level_model.load_all()

load sec_0
load sec_1
load sec_2
load sec_3
load sec_4


In [94]:
def h2_subset(h2df, files):
    h2sub = h2df[h2df['files'].isin(files)][list(set(h2df.columns) - {"files"})]
    h2sub.index = range(0, h2sub.shape[0])
    return h2sub

In [23]:
all_labels2 = pd.Series(dtype=object)
all_results2 = []

for chunk in tqdm.tqdm(ds.chunked(testdf_shuffled, 500)):
    x = ds.files_to_dataset(chunk['files'])
    label = chunk['category']
    firstdf = h2_subset(h2df, chunk['files'])
    resclasses = two_level_model.predict_arrs(x, firstdf)
    
    all_labels2 = all_labels2.append(label,  ignore_index=True)
    all_results2.extend(resclasses)

100%|██████████| 1/1 [00:08<00:00,  8.18s/it]


In [24]:
print("Number of test images: {}\nAccuracy: {}".format(len(all_labels2), float(sum(all_labels2 == all_results2))/len(all_labels2)))

Number of test images: 5
Accuracy: 0.4


Save the results as pickle.

In [25]:
results_df2 = pd.DataFrame({"labels": all_labels2, "prediction": all_results2})

In [26]:
results_df2.to_pickle("results/twolevelmodel_prediction_fgvc.dat")

### Normalized Class Accuracy

https://arxiv.org/pdf/1306.5151.pdf

In [106]:
# twolevelmodel_prediction.dat
results_df1 = pd.read_pickle("results/baseline_model_prediction_fgvc.dat")
results_df2 = pd.read_pickle("results/twolevelmodel_prediction_fgvc.dat")


In [117]:
categories = [str(i) for i in range(0, 100)]

In [21]:
def calc_tp(df, cat):
    trs = (df["labels"] == cat)
    prds = (df["prediction"] == cat)
    return sum(trs & prds)

In [223]:
TP2 = {cat: calc_tp(results_df2, cat) for cat in categories}

In [232]:
TP1 = {cat: calc_tp(results_df1, cat) for cat in categories}

In [233]:
sum(TP1.values())

245

In [234]:
sum(TP2.values())

354

In [236]:
100*sum(TP2.values())/sum(TP1.values())

144.48979591836735

In [224]:
def calc_Cpp(df, cat):
    return calc_tp(df, cat)/sum(df["labels"]==cat)

In [225]:
cna2_list = [calc_Cpp(results_df2, cat) for cat in categories]

In [226]:
np.mean(cna2_list)

0.10616755793226382

In [230]:
cna1_list = [calc_Cpp(results_df1, cat) for cat in categories]

In [231]:
np.mean(cna1_list)

0.073556149732620313